In [31]:
from random import randint

def get_arr(size):
    arr = [ randint(0,10) for _ in range(size)]
    return arr

def get_sorting_arr(get_arr):
    return sorted(get_arr(15))

def get_targer(arr):
    index = len(arr) - 1
    x_index = randint(0, index)
    return arr[x_index]

# arr = get_sorting_arr(get_arr)

linear search

In [37]:
def linear_search(arr:list, serch_element):
    for i in range(len(arr)):
        if arr[i] == serch_element:
            return arr[i]
    return -1

binary_search

In [36]:
def binary_search(arr:list, x):
    low = 0
    hight = len(arr) - 1
    mid = 0

    while low <= hight:
        
        mid = (hight + low) // 2

        #если х больше за значение по средние списка, игнорируем левую половину
        if arr[mid] < x:
            low = mid + 1
        # елси я меньше за значение по средине, игнорируем правую половину  
        elif arr[mid] > x:
            hight = mid - 1
        
        else:
            return mid
    return -1

arr = get_sorting_arr(get_arr)
print(arr)
element = get_targer(arr)
result = binary_search(arr, element)
result

[0, 1, 1, 2, 3, 5, 6, 7, 7, 8, 9, 9, 9, 10, 10]


3

index_linear search

In [107]:
from pydantic import BaseModel, Field, ValidationError
from typing import Union

class InputData(BaseModel):
    """модель для валидации входного массива и шага"""
    array: list[int] = Field(
        ...,  # поле обезательное для заполения
        min_length=1, 
        description='отсорированный список чисел'
        )
    step: int = Field(
        ...,
        gt=0, #больше чем (greater than )
        description='шаг для создания индекстной таблицы'
    )

class BinarySearchResult(BaseModel):
    """модель для валидации результата бинарного поиска"""
    result: Union[int, tuple[int, int]] #индекс или диапазон

class SearchInput(BaseModel):
    """валидация данных для поиска"""
    array: list[int] = Field(
        ..., 
        min_length=1, 
        description='отсорированный список чисел'
        )
    index_tuple: list[tuple[int, int]] = Field(
        ...,
        description='индексная таблица'
    )
    target: int = Field(
        ...,
        description='искомый елемент'
    ) 

#основной отсорированный массив
arr = get_sorting_arr(get_arr)

IT = list[tuple[int, int]]
def create_index_table(array, step)->IT:
    """
    создать индексную таблицу

    array: list[int] - изначальный отсортированный список
    step: int - шаг через выборки елементов из array

    example:
        arr = [0, 0, 2, 4, 5, 5, 6, 7, 9, 10]
        step = 4
        ------------
        append (0,0) # (A,B) A-елемент B-индекс этого елемента
        step_4
        append(5,4) # (A,B) A-елемент B-индекс этого елемента
        step_4
        append(9,8) # (A,B) A-елемент B-индекс этого елемента
    return:
        [(0, 0), (5, 4), (9, 8)]
    *числа в изначалном масиве должны быть равномерно розмещены
    *найти оптимальный шаг чтоб покрыть изначальный массив
    """
    index_table = []
    for i in range(0, len(array), step):
        index_table.append((array[i], i))
    return index_table

index_table = create_index_table(arr, 4)

def binary_search(index_table,target)->int|tuple[int,int]:
    """
    поиск искомого елемента в индексной таблице

    Args:
        index_table:list[tuple(int,int)] - индексная таблица 
            из create_index_table
        targer:int - искомый елемент

    Returned:
        есть всего два варианта
        1. искомый елемент найден, возврат индекса искомого елемента
        2. искомый индес не найден, возврат start диапазона

    Exemple:
        *выполнение функции create_index_table
            [0, 0, 2, 4, 5, 5, 6, 7, 9, 10] - изначальный список
            *укоротили таблицу в 4 раза
            [(0, 0), (5, 4), (9, 8)]
        ** конец выполнения функции

        [(x,y), (x,y), (x,y), (x,y), (x,y)]
                        |   
                    найти средину списка
                        |
                    проверить нашли ли targer, 
                        если target > y,
                [(x,y), (x,y), (x,y), (x,y), (x,y)]
                                        |
                                        ищем средину средины
                                        |
                            *повторяем все действия,
                            пока start >= end
                        если target < y,
                [(x,y), (x,y), (x,y), (x,y), (x,y)]
                        |
                        сдвинуться влево найти средину средины,
                        повторить оставшейся шаги
    """
    start = 0
    end = len(index_table) - 1

    while start <= end:
        mid = (start + end) // 2

        if index_table[mid][0] == target:
            return index_table[mid][1] #нашли искомый елемент
        
        elif index_table[mid][0] < target:
            start = mid + 1

        else:
            end = mid - 1

    return (start, end) # не нашли искомый елемент

def _get_range(start:int, index_table:IT, array:int)->tuple[int,int]:
    """
    определение диапазона линейного поиска

    exemple:
        start == 0, искомый елмент был меньше выбраных елементов
            в бинарном поиске. Предположительно находиться в начале списка
        start >= len(index_table), искомый елемент был больше выбраных елементов
            в бинарном поиске. Предположительно находится в конце списка
        в противнос случа нужно искать между диапазонами выше
    """

    if start == 0:
        range_start, range_end = (0, index_table[0][1])

    elif start >= len(index_table):
        range_start, range_end = (index_table[-1][1], len(array))

    else:
        range_start, range_end = (index_table[start-1][1], index_table[start][1])

    return range_start, range_end

def _bruteforce(array:list, range_start:int, range_end:int, target:int):
    """
    перебор елементов

    Args:
        array:list - список по которому будет поиск
        range_start:int - старт начала диапазона
        range_end:int - конец диапазона
    """
    for i in range(range_start, range_end):
        if array[i] == target:
            return i #вернуть идекс искомого елемента
    
def index_sequential_searsh(array, index_table, target)->int:
    """
    функция поиска

    array: list[int] - изначальный отсортированный список
    index_table: list[tuple(int,int)] - индексная таблица из create_index_table
    targer:int - искомый елемент

    Returned:int
        -1 - елемент не найден
        int:any - индекс искомого елемента
        
    """
    result = binary_search(index_table, target)

    if isinstance(result, int):
        return result #вернуть индекс искомого елемента
    
    start, _ = result

    range_start, range_end = _get_range(start, index_table, array)
   
    result = _bruteforce(array, range_start, range_end, target)

    if result is not None:
        return result
        
    return -1 # если елемент не найден

if __name__ == '__main__':
    try:
        input_data = InputData(array=arr, step=4)
        index_table = create_index_table(
            input_data.array, 
            input_data.step
            )
        target = get_targer(arr)
        search_input = SearchInput(
            array=arr, 
            index_tuple=index_table, 
            target=target
            )
        result = index_sequential_searsh(
            search_input.array, 
            search_input.index_tuple, 
            search_input.target
            )
        print(f'arr: {arr}')
        print(f'index table: {index_table}')
        print(f'nead find: {target}')
        print(f'result searshing {result}')
    except ValidationError as e:
        print("Ошибка валидации данных:", e)


arr: [0, 0, 0, 1, 2, 2, 5, 5, 5, 6, 8, 9, 9, 10, 10]
index table: [(0, 0), (2, 4), (5, 8), (9, 12)]
nead find: 2
result searshing 4
